In [1]:
import numpy as np 
import pandas as pd 
import tqdm
import string 
from sklearn.svm import LinearSVC 
from sklearn.calibration import CalibratedClassifierCV 

from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes


In [4]:
import glob
import pandas as pd
from tensorflow import keras
import numpy as np
import os 
from sklearn.model_selection import StratifiedKFold, KFold
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, BatchNormalization, concatenate, Input, ConvLSTM2D, Reshape, Conv3D, Flatten, LSTM, GRU, Dense,Dropout, Add
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalMaxPool1D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Sequential, load_model
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import re 
from catboost import CatBoostClassifier


## Load data

In [9]:
train = pd.read_csv('./storage/motion_detect/train_features.csv') 
train_labels = pd.read_csv('./storage/motion_detect/train_labels.csv') 
test = pd.read_csv('./storage/motion_detect/test_features.csv') 
submission = pd.read_csv('./storage/motion_detect/sample_submission.csv')

## Preprocess Inputs

In [10]:
features = ['id', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z']
X_train = train[features].groupby('id').agg(['max', 'min', 'mean'])
X_test = test[features].groupby('id').agg(['max', 'min', 'mean'])

In [15]:
y_train = train_labels['label']

In [16]:
X_train.shape, X_test.shape, y_train.shape

((3125, 18), (782, 18), (3125,))

In [21]:
k = 10 
models = [] 
kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 20210205) 
for n_fold, (train_idx, val_idx) in enumerate(kfold.split(X_train, y_train)): 
    train_x, val_x = X_train.loc[train_idx,:], X_train.loc[val_idx,:]
    train_y, val_y = y_train.loc[train_idx], y_train.loc[val_idx] 
    
    model = CatBoostClassifier(iterations = 10000, 
                               learning_rate = 0.01, 
                               l2_leaf_reg = 3.5,
                               depth = 8, 
                               loss_function = 'MultiClass',
                               eval_metric = 'MultiClass',
                               use_best_model = True, 
                               task_type = 'GPU',
                               verbose = 200,
                               random_seed = 88888,
                               classes_count=61)  
    
    
    model.fit(train_x, train_y, eval_set = (val_x, val_y)) 
    models.append(model)


0:	learn: 3.9969269	test: 4.0075726	best: 4.0075726 (0)	total: 74.9ms	remaining: 12m 28s
200:	learn: 1.6819033	test: 1.8450783	best: 1.8450783 (200)	total: 11.6s	remaining: 9m 27s
400:	learn: 1.2503690	test: 1.5074055	best: 1.5074055 (400)	total: 23.4s	remaining: 9m 19s
600:	learn: 1.0117837	test: 1.3489592	best: 1.3489592 (600)	total: 35.6s	remaining: 9m 16s
800:	learn: 0.8425585	test: 1.2545096	best: 1.2545096 (800)	total: 47.9s	remaining: 9m 10s
1000:	learn: 0.7143644	test: 1.1918201	best: 1.1918201 (1000)	total: 1m	remaining: 9m
1200:	learn: 0.6156801	test: 1.1451321	best: 1.1451321 (1200)	total: 1m 12s	remaining: 8m 49s
1400:	learn: 0.5373063	test: 1.1098325	best: 1.1098325 (1400)	total: 1m 24s	remaining: 8m 37s
1600:	learn: 0.4731040	test: 1.0828490	best: 1.0828490 (1600)	total: 1m 36s	remaining: 8m 25s
1800:	learn: 0.4202208	test: 1.0611828	best: 1.0611828 (1800)	total: 1m 47s	remaining: 8m 10s
2000:	learn: 0.3754311	test: 1.0439918	best: 1.0439918 (2000)	total: 1m 59s	remaining

In [22]:
pred1 = models[0].predict_proba(X_test) 
pred2 = models[1].predict_proba(X_test) 
pred3 = models[2].predict_proba(X_test) 
pred4 = models[3].predict_proba(X_test) 
pred5 = models[4].predict_proba(X_test)
pred6 = models[5].predict_proba(X_test) 
pred7 = models[6].predict_proba(X_test) 
pred8 = models[7].predict_proba(X_test) 
pred9 = models[8].predict_proba(X_test) 
pred10 = models[9].predict_proba(X_test)

In [26]:
pred_avg = (pred1 + pred2 + pred3 + pred4 + pred5 + pred6 + pred7 + pred8 + pred9 + pred10)/10.0 

In [29]:
submission.iloc[:,1:] = pred_avg

In [30]:
submission.head()

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0.000274,0.000080,0.000225,0.000153,0.000421,0.000065,0.000118,0.000161,0.000391,...,0.010140,0.005712,0.000985,0.000202,0.000223,0.000148,0.000635,0.077717,0.000078,0.000802
1,3126,0.001259,0.000081,0.000060,0.000300,0.000258,0.000184,0.000052,0.000307,0.000209,...,0.001246,0.000056,0.000201,0.000105,0.000146,0.000158,0.000266,0.000099,0.000083,0.000257
2,3127,0.004029,0.565608,0.000647,0.006835,0.000398,0.001483,0.043298,0.021876,0.002496,...,0.000525,0.000385,0.000799,0.021989,0.003096,0.011984,0.000295,0.000324,0.000934,0.003002
3,3128,0.001720,0.000509,0.003320,0.000779,0.000111,0.000132,0.000052,0.001607,0.000321,...,0.000163,0.000123,0.000275,0.000212,0.001370,0.000051,0.002375,0.000126,0.000048,0.036578
4,3129,0.000649,0.000018,0.000031,0.000206,0.000105,0.000120,0.000021,0.000057,0.000333,...,0.000034,0.000013,0.000039,0.000012,0.000056,0.000048,0.000154,0.000027,0.000032,0.000096


In [31]:
submission.to_csv("catboost_min_max_mean.csv",index=False) 

In [32]:
bi_gru = pd.read_csv('bi_gru_5_fold.csv') 

In [41]:
submission.iloc[:,1:] = (bi_gru.iloc[:,1:] + submission.iloc[:,1:])/2

In [42]:
submission.head()

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0.000209,0.006730,0.000117,0.000079,0.000230,0.000034,0.079407,0.001780,0.000196,...,0.005358,0.002903,0.000673,0.000197,0.000112,0.000077,0.000464,0.098014,0.000053,0.000402
1,3126,0.000699,0.000048,0.000031,0.000172,0.000139,0.000102,0.000026,0.000154,0.000105,...,0.000623,0.000028,0.000103,0.000053,0.000074,0.000088,0.000154,0.000050,0.000045,0.000163
2,3127,0.002101,0.283705,0.000378,0.003429,0.000341,0.000757,0.028461,0.011115,0.001299,...,0.001368,0.009519,0.000456,0.012402,0.001591,0.006001,0.000300,0.006150,0.000482,0.006662
3,3128,0.001027,0.000259,0.001679,0.000398,0.000057,0.000080,0.000034,0.000807,0.000209,...,0.000117,0.000064,0.000141,0.000290,0.000705,0.000033,0.004485,0.000125,0.000025,0.019391
4,3129,0.021127,0.013242,0.000036,0.000205,0.025316,0.000223,0.000020,0.000264,0.000188,...,0.000027,0.000007,0.000048,0.000035,0.000036,0.000035,0.004222,0.000076,0.001445,0.000206


In [43]:
submission.to_csv('gru_catboost_average.csv',index=False)